# Proyecto Final Bootcamp: Análisis de Leads

## 1. Introducción

En este notebook se realiza un análisis completo de un dataset de leads con el objetivo de explorar los datos, preparar un modelo de Machine Learning y proponer futuras líneas de investigación.

El objetivo principal es predecir la conversión de un lead a cliente a partir de sus características y acciones previas. Para ello, se seguirán los pasos clásicos de la ciencia de datos: exploración, limpieza, modelado, evaluación e interpretación de resultados.

## 2. Carga y descripción del dataset

Se utiliza un dataset ficticio/simulando leads comerciales, preparado para prácticas de aprendizaje. - [Lead Scoring Dataset – Kaggle](https://www.kaggle.com/datasets/amritachatterjee09/lead-scoring-dataset/code)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split, validation_curve, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    confusion_matrix, ConfusionMatrixDisplay, classification_report, 
    roc_curve, roc_auc_score, precision_recall_curve, average_precision_score
)
import xgboost as xgb
import shap
import joblib

In [ ]:
df = pd.read_csv('/Users/fernandoegusquizamartin/Documents/Código /Proyecto Final/Lead scoring/Data Set Lead Scoring/daraset/Lead Scoring.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.describe(include='all').T

## 3. Análisis Exploratorio de Datos (EDA)

Vamos a analizar visualmente la distribución de las variables principales y la relación con la conversión.

In [ ]:
plt.figure(figsize=(4,3))
sns.countplot(data=df, x='Converted')
plt.title('Distribución de la variable objetivo: Converted')
plt.xlabel('Convertido (1) / No convertido (0)')
plt.ylabel('Cantidad')
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
sns.histplot(df['TotalVisits'], kde=True, bins=30)
plt.title('Distribución Total de Visitas')
plt.xlabel('Total Visits')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
sns.histplot(df['Total Time Spent on Website'], kde=True, bins=30)
plt.title('Distribución de Tiempo Total en la Web')
plt.xlabel('Total Time Spent on Website')
plt.ylabel('Frecuencia')
plt.show()

plt.figure(figsize=(6,4))
sns.histplot(df['Page Views Per Visit'], kde=True, bins=30)
plt.title('Distribución de Page Views Per Visit')
plt.xlabel('Page Views Per Visit')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
num_cols = df.select_dtypes(include=[np.number])
corr = num_cols.corr()

plt.figure(figsize=(8,6))
sns.heatmap(corr, annot=True, cmap='Blues', fmt='.2f')
plt.title('Matriz de Correlaciones')
plt.show()

### 3.1. Principales conclusiones del EDA
- Hay cierta desbalance en la variable objetivo (`Converted`).
- Variables como visitas y tiempo en la web tienen colas largas (outliers).
- Algunas variables numéricas están correlacionadas.
- Habrá que transformar o tratar algunas variables para el modelo.

## 4. Preparación y limpieza de datos

En esta sección:
- Eliminamos columnas irrelevantes o con muchos valores únicos (IDs).
- Tratamos valores nulos y outliers.
- Codificamos variables categóricas.


In [ ]:
# Elimo columnas no informativas y que puedan afectar sobre la predicción de calidad del lead
columnas_a_eliminar = [
    'Prospect ID',
    'Lead Number',
    'Receive More Updates About Our Courses',
    'Lead Quality',
    'Update me on Supply Chain Content',
    'I agree to pay the amount through cheque',
    'Last Notable Activity',
    'A free copy of Mastering The Interview',
    'Asymmetrique Profile Score',
    'Asymmetrique Activity Score',
    'Asymmetrique Activity Index',
    'Asymmetrique Profile Index',
    'Last Activity',
    'Lead Profile',
    'Tags'
]

df = df.drop(columns=columnas_a_eliminar, errors='ignore')

df.head()


In [ ]:
# Convertir valores "select" en nulos, para evitar que influyan.
df = df.replace("Select", np.nan)

In [ ]:
# Revisar nulos
df.isnull().sum()

In [ ]:
# Cambio nulos con la moda
for col in df.columns:
    if df[col].isnull().sum() > 0:
        df[col].fillna(df[col].mode()[0], inplace=True)


In [ ]:
# Outliers: Hago un capado sencillo usando percentiles
num_cols = df.select_dtypes(include=[np.number]).columns
for col in num_cols:
    q_low = df[col].quantile(0.01)
    q_hi  = df[col].quantile(0.99)
    df[col] = np.clip(df[col], q_low, q_hi)

In [ ]:
# Codificación de variables categóricas (one-hot encoding)
cat_cols = df.select_dtypes(include='object').columns
df = pd.get_dummies(df, columns=cat_cols, drop_first=True)
df.head()

## 5. Modelado: Random Forest Classifier

Entrenamos un modelo básico de Random Forest para predecir la conversión del lead.

In [ ]:
X = df.drop('Converted', axis=1)
y = df['Converted']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:,1]
print('Classification Report:')
print(classification_report(y_test, y_pred))
print('ROC-AUC:', roc_auc_score(y_test, y_proba))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Matriz de confusión')
plt.show()

### 5.1. Importancia de variables
Veamos qué variables han sido más relevantes para el modelo:

In [ ]:
importances = model.feature_importances_
feat_names = X.columns
feat_imp = pd.Series(importances, index=feat_names).sort_values(ascending=False)
plt.figure(figsize=(8,5))
feat_imp.head(10).plot(kind='bar')
plt.title('Top 10 variables más importantes')
plt.ylabel('Importancia')
plt.show()

### 5.2. Interpretación del modelo con SHAP
Explicamos las predicciones usando SHAP para ver el peso de cada variable en la predicción.

In [ ]:
import shap


In [ ]:
modelo = RandomForestClassifier()
modelo.fit(X_train, y_train)

In [ ]:
explainer = shap.TreeExplainer(modelo)
shap_values = explainer.shap_values(X_test)


In [ ]:
print(type(shap_values))
if isinstance(shap_values, list):
    print([v.shape for v in shap_values])
else:
    print(shap_values.shape)
print(X_test.shape)


In [ ]:
shap.summary_plot(shap_values[:, :, 1], X_test)

## 6. Prueba del modelo


In [ ]:
import joblib

# 1. Entrenamos el modelo con todos los datos disponibles
X_full = df.drop('Converted', axis=1)
y_full = df['Converted']

modelo_final = RandomForestClassifier(n_estimators=100, random_state=42)
modelo_final.fit(X_full, y_full)

# 2. Guardamos el modelo entrenado
joblib.dump(modelo_final, 'modelo_lead_scoring.pkl')

# Guardamos la lista de columnas 
columnas_modelo = X_full.columns.tolist()
joblib.dump(columnas_modelo, 'columnas_modelo.pkl')

print("Modelo final y columnas guardados correctamente.")

# 3. Función para predecir la probabilidad de conversión de un nuevo lead

def predecir_lead(nuevo_lead, modelo, columnas_modelo):
    """
    nuevo_lead: dict con las características del nuevo lead (mismo formato que las columnas originales)
    modelo: modelo entrenado
    columnas_modelo: lista de columnas que espera el modelo (incluyendo dummies)
    """
    lead_df = pd.DataFrame([nuevo_lead])
    lead_df = pd.get_dummies(lead_df)
    # Añadir columnas faltantes y ordenar igual que el modelo
    for col in columnas_modelo:
        if col not in lead_df.columns:
            lead_df[col] = 0
    lead_df = lead_df[columnas_modelo]
    # Predicción
    prob = modelo.predict_proba(lead_df)[:, 1][0]
    return prob

# 4. Ejemplos prácticos de uso

# Ejemplo 1: Lead muy interesado (valores altos)
nuevo_lead_1 = {
    'TotalVisits': 8,
    'Total Time Spent on Website': 400,
    'Page Views Per Visit': 6,
    # ...añade aquí más campos relevantes de tu modelo...
    # Incluye las columnas dummies que sean importantes según tu preprocesado
}

# Ejemplo 2: Lead poco activo (valores bajos)
nuevo_lead_2 = {
    'TotalVisits': 1,
    'Total Time Spent on Website': 50,
    'Page Views Per Visit': 1,
    # ...añade aquí más campos relevantes de tu modelo...
}

# Ejemplo 3: Lead sin información en campos clave
nuevo_lead_3 = {
    'TotalVisits': 2,
    'Total Time Spent on Website': 60,
    'Page Views Per Visit': 1.5,
    # ...campos faltantes se rellenarán como 0 automáticamente...
}

# Predicción
prob1 = predecir_lead(nuevo_lead_1, modelo_final, columnas_modelo)
prob2 = predecir_lead(nuevo_lead_2, modelo_final, columnas_modelo)
prob3 = predecir_lead(nuevo_lead_3, modelo_final, columnas_modelo)

print(f"Lead 1 - Probabilidad de conversión: {prob1:.2%}")
print(f"Lead 2 - Probabilidad de conversión: {prob2:.2%}")
print(f"Lead 3 - Probabilidad de conversión: {prob3:.2%}")


In [ ]:
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, roc_auc_score

# 1. Carga del modelo y las columnas del modelo
modelo = joblib.load('modelo_lead_scoring.pkl')
columnas_modelo = joblib.load('columnas_modelo.pkl')

# 2. Carga del archivo CSV
df_nuevos = pd.read_csv('/Users/fernandoegusquizamartin/Documents/Código /Proyecto Final/Lead scoring/Data Set Lead Scoring/daraset/eval_leads.csv')

# 3. Preprocesamiento
df_nuevos = df_nuevos.replace("Select", np.nan)
columnas_a_eliminar = [
    'Prospect ID',
    'Lead Number',
    'Receive More Updates About Our Courses',
    'Lead Quality',
    'Update me on Supply Chain Content',
    'I agree to pay the amount through cheque',
    'Last Notable Activity',
    'A free copy of Mastering The Interview',
    'Asymmetrique Profile Score',
    'Asymmetrique Activity Score',
    'Asymmetrique Activity Index',
    'Asymmetrique Profile Index',
    'Last Activity',
    'Lead Profile',
    'Tags'
]
converted_original = df_nuevos['Converted'].copy() if 'Converted' in df_nuevos.columns else None

df_nuevos = df_nuevos.drop(columns=columnas_a_eliminar, errors='ignore')
df_nuevos_dummies = pd.get_dummies(df_nuevos)
for col in columnas_modelo:
    if col not in df_nuevos_dummies.columns:
        df_nuevos_dummies[col] = 0
df_nuevos_dummies = df_nuevos_dummies[columnas_modelo]

# 4. Predicción del modelo (probabilidad y predicción binaria)
df_nuevos['Prediccion_Modelo_Prob'] = modelo.predict_proba(df_nuevos_dummies)[:, 1]
df_nuevos['Prediccion_Modelo'] = (df_nuevos['Prediccion_Modelo_Prob'] >= 0.5).astype(int)

# 5. Comparación con la variable Converted

# Limpieza: sólo filas con ambos valores no nulos
filtro = ~converted_original.isnull()
df_compara = df_nuevos[filtro].copy()
df_compara['Converted'] = converted_original[filtro].astype(int)

print(df_compara[['Converted', 'Prediccion_Modelo', 'Prediccion_Modelo_Prob']].head())

# Matriz de confusión y métricas
cm = confusion_matrix(df_compara['Converted'], df_compara['Prediccion_Modelo'])
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['No convertido', 'Convertido'])
disp.plot(cmap='Blues')
plt.title('Matriz de confusión: Converted vs Predicción del modelo')
plt.show()

print("Classification Report:\n")
print(classification_report(df_compara['Converted'], df_compara['Prediccion_Modelo']))

# ROC-AUC
roc_auc = roc_auc_score(df_compara['Converted'], df_compara['Prediccion_Modelo_Prob'])
print(f'ROC-AUC: {roc_auc:.3f}')

# algunos casos donde discrepan
print("\nEjemplos donde modelo y real no coinciden:")
print(df_compara[df_compara['Converted'] != df_compara['Prediccion_Modelo']][
    ['Converted', 'Prediccion_Modelo', 'Prediccion_Modelo_Prob']].head(10))


El modelo predice correctamente un 67% de los leads. Tiene más aciertos que un modelo aleatorio o uno que siempre prediga la clase mayoritaria. Sin embargo, aún comete algunos errores, sobre todo al predecir algunos leads como ‘convertidos’ que finalmente no lo son. Según el objetivo del negocio, podríamos ajustar el umbral para priorizar no perder oportunidades (recall) o para minimizar falsos positivos (precisión).

In [ ]:
# Carga dataset de entrenamiento
df = pd.read_csv('/Users/fernandoegusquizamartin/Documents/Código /Proyecto Final/Lead scoring/Data Set Lead Scoring/daraset/Lead Scoring.csv')

# Reemplaza valores "Select" por nulos
df = df.replace("Select", np.nan)

# Elimina columnas innecesarias
columnas_a_eliminar = [
    'Prospect ID', 'Lead Number', 'Receive More Updates About Our Courses', 'Lead Quality',
    'Update me on Supply Chain Content', 'I agree to pay the amount through cheque',
    'Last Notable Activity', 'A free copy of Mastering The Interview',
    'Asymmetrique Profile Score', 'Asymmetrique Activity Score', 'Asymmetrique Activity Index',
    'Asymmetrique Profile Index', 'Last Activity', 'Lead Profile', 'Tags'
]
df = df.drop(columns=columnas_a_eliminar, errors='ignore')

# Opcional: Rellenar nulos (puedes personalizar esto según tu criterio)
for col in df.columns:
    if df[col].isnull().sum() > 0:
        df[col].fillna(df[col].mode()[0], inplace=True)

# Convertir variables categóricas a dummies
X = df.drop('Converted', axis=1)
y = df['Converted']
X = pd.get_dummies(X)

# Guarda las columnas para futuros datos de test
columnas_modelo = X.columns.tolist()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

In [ ]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
rf_probs = rf.predict_proba(X_test)[:,1]
rf_pred = rf.predict(X_test)

print("Random Forest Classification Report:")
print(classification_report(y_test, rf_pred))
print("Random Forest ROC-AUC: ", roc_auc_score(y_test, rf_probs))


In [ ]:
xgb_model = xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)
xgb_probs = xgb_model.predict_proba(X_test)[:,1]
xgb_pred = xgb_model.predict(X_test)

print("XGBoost Classification Report:")
print(classification_report(y_test, xgb_pred))
print("XGBoost ROC-AUC: ", roc_auc_score(y_test, xgb_probs))


In [ ]:
def plot_roc_pr(y_test, probs, modelo_name):
    # ROC
    fpr, tpr, _ = roc_curve(y_test, probs)
    auc_score = roc_auc_score(y_test, probs)
    plt.figure()
    plt.plot(fpr, tpr, label=f'ROC curve (AUC = {auc_score:.2f})')
    plt.plot([0,1],[0,1],'k--')
    plt.title(f'Curva ROC - {modelo_name}')
    plt.xlabel('Tasa de falsos positivos')
    plt.ylabel('Tasa de verdaderos positivos')
    plt.legend()
    plt.show()
    # PR
    precision, recall, _ = precision_recall_curve(y_test, probs)
    avg_precision = average_precision_score(y_test, probs)
    plt.figure()
    plt.plot(recall, precision, label=f'PR curve (AP = {avg_precision:.2f})')
    plt.title(f'Curva Precision-Recall - {modelo_name}')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.legend()
    plt.show()

plot_roc_pr(y_test, rf_probs, "Random Forest")
plot_roc_pr(y_test, xgb_probs, "XGBoost")

In [ ]:
explainer = shap.TreeExplainer(xgb_model)
shap_values = explainer.shap_values(X_test)
shap.summary_plot(shap_values, X_test)

In [ ]:
from sklearn.model_selection import validation_curve

param_range = [10, 50, 100, 200, 300]
train_scores, test_scores = validation_curve(
    RandomForestClassifier(random_state=42),
    X, y, param_name="n_estimators", param_range=param_range,
    cv=3, scoring="roc_auc"
)
plt.figure()
plt.plot(param_range, test_scores.mean(axis=1), label='Test ROC-AUC')
plt.plot(param_range, train_scores.mean(axis=1), label='Train ROC-AUC')
plt.xlabel("n_estimators")
plt.ylabel("ROC-AUC")
plt.title("Curva de validación - Random Forest")
plt.legend()
plt.show()

In [ ]:
param_grid = {
    'max_depth': [3, 5, 7],
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2]
}
grid_xgb = GridSearchCV(
    xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'),
    param_grid, cv=3, scoring='roc_auc'
)
grid_xgb.fit(X_train, y_train)
print("Mejores parámetros XGBoost:", grid_xgb.best_params_)
print("Mejor ROC-AUC (CV):", grid_xgb.best_score_)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, roc_auc_score

df_nuevos = pd.read_csv('/Users/fernandoegusquizamartin/Documents/Código /Proyecto Final/Lead scoring/Data Set Lead Scoring/daraset/eval_leads.csv')
df_nuevos = df_nuevos.replace("Select", np.nan)

columnas_a_eliminar = [
    'Prospect ID', 'Lead Number', 'Receive More Updates About Our Courses', 'Lead Quality',
    'Update me on Supply Chain Content', 'I agree to pay the amount through cheque',
    'Last Notable Activity', 'A free copy of Mastering The Interview',
    'Asymmetrique Profile Score', 'Asymmetrique Activity Score', 'Asymmetrique Activity Index',
    'Asymmetrique Profile Index', 'Last Activity', 'Lead Profile', 'Tags'
]
converted_original = df_nuevos['Converted'].copy() if 'Converted' in df_nuevos.columns else None

df_nuevos = df_nuevos.drop(columns=columnas_a_eliminar, errors='ignore')
df_nuevos_dummies = pd.get_dummies(df_nuevos)

for col in columnas_modelo:
    if col not in df_nuevos_dummies.columns:
        df_nuevos_dummies[col] = 0
df_nuevos_dummies = df_nuevos_dummies[columnas_modelo]

# 1. Predicción con Random Forest
df_nuevos['RF_Prob'] = rf.predict_proba(df_nuevos_dummies)[:, 1]
df_nuevos['RF_Pred'] = (df_nuevos['RF_Prob'] >= 0.5).astype(int)

# 2. Predicción con XGBoost
df_nuevos['XGB_Prob'] = xgb_model.predict_proba(df_nuevos_dummies)[:, 1]
df_nuevos['XGB_Pred'] = (df_nuevos['XGB_Prob'] >= 0.5).astype(int)

# 3. Comparación con variable real
filtro = ~converted_original.isnull()
df_compara = df_nuevos[filtro].copy()
df_compara['Converted'] = converted_original[filtro].astype(int)

print("\nHEAD de resultados comparativos:")
print(df_compara[['Converted', 'RF_Pred', 'RF_Prob', 'XGB_Pred', 'XGB_Prob']].head())

# 4. Métricas y matriz de confusión
for modelo, pred_col, prob_col, nombre in [
    (rf, 'RF_Pred', 'RF_Prob', 'Random Forest'),
    (xgb_model, 'XGB_Pred', 'XGB_Prob', 'XGBoost')
]:
    print(f"\n--- {nombre} ---")
    cm = confusion_matrix(df_compara['Converted'], df_compara[pred_col])
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['No convertido', 'Convertido'])
    disp.plot(cmap='Blues')
    plt.title(f'Matriz de confusión: Converted vs {nombre}')
    plt.show()

    print(classification_report(df_compara['Converted'], df_compara[pred_col]))
    roc_auc = roc_auc_score(df_compara['Converted'], df_compara[prob_col])
    print(f'{nombre} ROC-AUC: {roc_auc:.3f}')

    print("\nEjemplos donde modelo y real no coinciden:")
    print(df_compara[df_compara['Converted'] != df_compara[pred_col]][
        ['Converted', pred_col, prob_col]].head(10))

# 5. Curvas ROC y PR
from sklearn.metrics import roc_curve, precision_recall_curve, average_precision_score

# ROC
for prob_col, nombre in [('RF_Prob', 'Random Forest'), ('XGB_Prob', 'XGBoost')]:
    fpr, tpr, _ = roc_curve(df_compara['Converted'], df_compara[prob_col])
    auc = roc_auc_score(df_compara['Converted'], df_compara[prob_col])
    plt.plot(fpr, tpr, label=f"{nombre} (AUC={auc:.2f})")
plt.plot([0, 1], [0, 1], 'k--')
plt.title("Curva ROC - eval_leads.csv")
plt.xlabel("Falsos positivos")
plt.ylabel("Verdaderos positivos")
plt.legend()
plt.show()

# PR
for prob_col, nombre in [('RF_Prob', 'Random Forest'), ('XGB_Prob', 'XGBoost')]:
    precision, recall, _ = precision_recall_curve(df_compara['Converted'], df_compara[prob_col])
    ap = average_precision_score(df_compara['Converted'], df_compara[prob_col])
    plt.plot(recall, precision, label=f"{nombre} (AP={ap:.2f})")
plt.title("Curva Precision-Recall - eval_leads.csv")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend()
plt.show()


In [ ]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier

# --- Tuneo para Random Forest ---
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_rf = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid_rf,
    cv=3,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=2
)
grid_rf.fit(X_train, y_train)
print("Mejores parámetros Random Forest:", grid_rf.best_params_)
print("Mejor ROC-AUC (CV) Random Forest:", grid_rf.best_score_)

# --- Tuneo para XGBoost ---
param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.7, 1],
    'colsample_bytree': [0.7, 1],
    'gamma': [0, 1, 5]
}

grid_xgb = GridSearchCV(
    xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'),
    param_grid_xgb,
    cv=3,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=2
)
grid_xgb.fit(X_train, y_train)
print("Mejores parámetros XGBoost:", grid_xgb.best_params_)
print("Mejor ROC-AUC (CV) XGBoost:", grid_xgb.best_score_)


In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# 1. Split de datos
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# 2. Balanceo con SMOTE
sm = SMOTE(random_state=42)
X_train_bal, y_train_bal = sm.fit_resample(X_train, y_train)
print("Distribución tras SMOTE:", y_train_bal.value_counts())

# 3. Entrenamiento de modelos balanceados

# Random Forest con class_weight
from sklearn.ensemble import RandomForestClassifier
rf_bal = RandomForestClassifier(class_weight='balanced', random_state=42)
rf_bal.fit(X_train_bal, y_train_bal)

# XGBoost con scale_pos_weight
import xgboost as xgb
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
xgb_bal = xgb.XGBClassifier(
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss',
    scale_pos_weight=scale_pos_weight
)
xgb_bal.fit(X_train_bal, y_train_bal)


In [ ]:
# Preprocesado de eval_leads.csv 
df_nuevos = pd.read_csv('/Users/fernandoegusquizamartin/Documents/Código /Proyecto Final/Lead scoring/Data Set Lead Scoring/daraset/eval_leads.csv')
df_nuevos = df_nuevos.replace("Select", np.nan)
columnas_a_eliminar = [
    'Prospect ID', 'Lead Number', 'Receive More Updates About Our Courses', 'Lead Quality',
    'Update me on Supply Chain Content', 'I agree to pay the amount through cheque',
    'Last Notable Activity', 'A free copy of Mastering The Interview',
    'Asymmetrique Profile Score', 'Asymmetrique Activity Score', 'Asymmetrique Activity Index',
    'Asymmetrique Profile Index', 'Last Activity', 'Lead Profile', 'Tags'
]
converted_original = df_nuevos['Converted'].copy() if 'Converted' in df_nuevos.columns else None

df_nuevos = df_nuevos.drop(columns=columnas_a_eliminar, errors='ignore')
df_nuevos_dummies = pd.get_dummies(df_nuevos)
for col in columnas_modelo:
    if col not in df_nuevos_dummies.columns:
        df_nuevos_dummies[col] = 0
df_nuevos_dummies = df_nuevos_dummies[columnas_modelo]

# Predicción con modelos balanceados
df_nuevos['RF_Bal_Prob'] = rf_bal.predict_proba(df_nuevos_dummies)[:, 1]
df_nuevos['RF_Bal_Pred'] = (df_nuevos['RF_Bal_Prob'] >= 0.5).astype(int)

df_nuevos['XGB_Bal_Prob'] = xgb_bal.predict_proba(df_nuevos_dummies)[:, 1]
df_nuevos['XGB_Bal_Pred'] = (df_nuevos['XGB_Bal_Prob'] >= 0.5).astype(int)


In [ ]:
# Comparación con la variable real
filtro = ~converted_original.isnull()
df_compara = df_nuevos[filtro].copy()
df_compara['Converted'] = converted_original[filtro].astype(int)

print("\nHEAD de resultados comparativos (balanceados):")
print(df_compara[['Converted', 'RF_Bal_Pred', 'RF_Bal_Prob', 'XGB_Bal_Pred', 'XGB_Bal_Prob']].head())

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, roc_auc_score, roc_curve, precision_recall_curve, average_precision_score
import matplotlib.pyplot as plt

for modelo, pred_col, prob_col, nombre in [
    (rf_bal, 'RF_Bal_Pred', 'RF_Bal_Prob', 'Random Forest Balanceado'),
    (xgb_bal, 'XGB_Bal_Pred', 'XGB_Bal_Prob', 'XGBoost Balanceado')
]:
    print(f"\n--- {nombre} ---")
    cm = confusion_matrix(df_compara['Converted'], df_compara[pred_col])
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['No convertido', 'Convertido'])
    disp.plot(cmap='Blues')
    plt.title(f'Matriz de confusión: Converted vs {nombre}')
    plt.show()

    print(classification_report(df_compara['Converted'], df_compara[pred_col]))
    roc_auc = roc_auc_score(df_compara['Converted'], df_compara[prob_col])
    print(f'{nombre} ROC-AUC: {roc_auc:.3f}')

    print("\nEjemplos donde modelo y real no coinciden:")
    print(df_compara[df_compara['Converted'] != df_compara[pred_col]][
        ['Converted', pred_col, prob_col]].head(10))

# Curvas ROC y PR
# ROC
for prob_col, nombre in [('RF_Bal_Prob', 'Random Forest Balanceado'), ('XGB_Bal_Prob', 'XGBoost Balanceado')]:
    fpr, tpr, _ = roc_curve(df_compara['Converted'], df_compara[prob_col])
    auc = roc_auc_score(df_compara['Converted'], df_compara[prob_col])
    plt.plot(fpr, tpr, label=f"{nombre} (AUC={auc:.2f})")
plt.plot([0, 1], [0, 1], 'k--')
plt.title("Curva ROC - Modelos Balanceados (eval_leads.csv)")
plt.xlabel("Falsos positivos")
plt.ylabel("Verdaderos positivos")
plt.legend()
plt.show()

# PR
for prob_col, nombre in [('RF_Bal_Prob', 'Random Forest Balanceado'), ('XGB_Bal_Prob', 'XGBoost Balanceado')]:
    precision, recall, _ = precision_recall_curve(df_compara['Converted'], df_compara[prob_col])
    ap = average_precision_score(df_compara['Converted'], df_compara[prob_col])
    plt.plot(recall, precision, label=f"{nombre} (AP={ap:.2f})")
plt.title("Curva Precision-Recall - Modelos Balanceados (eval_leads.csv)")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend()
plt.show()


In [ ]:
probs = rf.predict_proba(X_test)[:, 1]

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np

umbrales = np.linspace(0, 1, 1000)
falsos_positivos = []
mejor_umbral = 1.0  

for umbral in umbrales:
    pred = (probs >= umbral).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()
    falsos_positivos.append(fp)
    if fp == 0:
        mejor_umbral = umbral
        break  

print(f"Umbral mínimo para 0 falsos positivos: {mejor_umbral:.3f}")


pred_0_fp = (probs >= mejor_umbral).astype(int)
print(confusion_matrix(y_test, pred_0_fp))


In [ ]:
probs = xgb_model.predict_proba(X_test)[:, 1]

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np

umbrales = np.linspace(0, 1, 1000)
falsos_positivos = []
mejor_umbral = 1.0  

for umbral in umbrales:
    pred = (probs >= umbral).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()
    falsos_positivos.append(fp)
    if fp == 0:
        mejor_umbral = umbral
        break  

print(f"Umbral mínimo para 0 falsos positivos: {mejor_umbral:.3f}")


pred_0_fp = (probs >= mejor_umbral).astype(int)
print(confusion_matrix(y_test, pred_0_fp))